# Condensed Phase-by-Phase Implementation Plan

## Phase 1: Environment Setup
- **Install Python 3.10+** and create an isolated environment (`venv` or `conda`).
- **Install required packages:** `requests`, `websockets`, `openai` (if needed).
- **Start LM Studio**, note its API endpoint (default: `http://localhost:1234/v1/chat/completions`).
- **Choose and verify Copilot output file location and format** (plain text or JSON).

## Phase 2: Communication Bridge
- Implement `query_lm_studio(prompt)` to send prompts via HTTP and return JSON.
- Add error handling for HTTP status codes and timeouts.
- Define a simple JSON message schema for agent requests and responses.
- Log all API interactions for debugging.

## Phase 3: Copilot Output Integration
- Write `read_copilot_output()` and `write_copilot_output(data)` functions.
- Test file I/O operations to ensure correct serialization (e.g., JSON parsing).
- Standardize file naming conventions and directory structure.
- Handle file permission errors and path validations.

## Phase 4: Orchestrator Agent
- Create an `OrchestratorAgent` class with:
  - `add_step(description, action)`
  - `run_next_step()`
- Maintain `steps`, `current_step`, and a persistent state file for recovery.
- Implement basic logging of each step’s input, output, and status.
- Prepare branching logic for conditional retries and error recovery.

## Phase 5: Basic Workflow Demo
- Instantiate the agent.
- Add steps:
  - Query LM Studio for code generation.
  - Read Copilot output.
  - (Optional) Validate or transform results.
- Loop through all steps with `run_next_step()`.
- Print or persist each output for review.

## Phase 6: Review & Next Steps
- Extend error handling with retry policies and fallback actions.
- Build a lightweight UI or CLI for step approval and feedback.
- Define success metrics (linting, test coverage, workflow latency).
- Integrate additional agents (test runners, deployment bots) and multi-language support.

## Phase 1: Environment Setup — Complete
- Python 3.11.9 virtual environment configured.
- Installed packages: `requests`, `websockets`, `openai`.
- Ready for LM Studio integration and file location setup.

In [ ]:
### Copilot Output File Location Selected
- Path: `c:\Users\tyler\source\repos\Wulfic\Orchestrator\copilot_output.json`
- Format: JSON

In [ ]:
import requests
import json
import logging
import time
from typing import Optional

# Configure logging
logging.basicConfig(filename='lm_studio_api.log', level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

LM_STUDIO_ENDPOINT = "http://127.0.0.1:1234/v1/chat/completions"

def query_lm_studio(prompt, model="oh-dcft-v3.1-claude-3-5-sonnet-20241022", timeout=30, max_retries=3, retry_delay=2):
    """Send a prompt to LM Studio and return the JSON response. Retries on failure."""
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }
    for attempt in range(1, max_retries + 1):
        try:
            response = requests.post(LM_STUDIO_ENDPOINT, headers=headers, data=json.dumps(payload), timeout=timeout)
            response.raise_for_status()
            logging.info(f"Prompt sent: {prompt}")
            logging.info(f"Response: {response.text}")
            return response.json()
        except requests.exceptions.Timeout:
            logging.error(f"Timeout occurred for prompt: {prompt} (Attempt {attempt})")
            if attempt < max_retries:
                time.sleep(retry_delay)
            else:
                return {"error": "timeout"}
        except requests.exceptions.RequestException as e:
            logging.error(f"HTTP error: {e} (Attempt {attempt})")
            if attempt < max_retries:
                time.sleep(retry_delay)
            else:
                return {"error": str(e)}
        except Exception as e:
            logging.error(f"Unexpected error: {e} (Attempt {attempt})")
            if attempt < max_retries:
                time.sleep(retry_delay)
            else:
                return {"error": str(e)}

### Next: Define a Simple JSON Message Schema
- Agent requests and responses will use a standardized JSON format for communication.
- Example schema:
```json
{
  "request": {
    "prompt": "...",
    "metadata": {
      "timestamp": "...",
      "agent_id": "..."
    }
  },
  "response": {
    "output": "...",
    "status": "success",
    "error": null
  }
}
```

In [ ]:
import time
from typing import Optional
def build_agent_request(prompt: str, agent_id: str = "orchestrator") -> dict:
    return {
        "request": {
            "prompt": prompt,
            "metadata": {
                "timestamp": time.strftime('%Y-%m-%d %H:%M:%S'),
                "agent_id": agent_id
            }
        }
    }

def build_agent_response(output: str, status: str = "success", error: Optional[str] = None) -> dict:
    return {
        "response": {
            "output": output,
            "status": status,
            "error": error
        }
    }

In [ ]:
# Test agent request/response schema and LM Studio query
sample_prompt = "What is the capital of France?"
request_obj = build_agent_request(sample_prompt)
print("Request object:", request_obj)

response_json = query_lm_studio(sample_prompt)
response_obj = build_agent_response(
    output=response_json.get('choices', [{}])[0].get('message', {}).get('content', "No output"),
    status="success" if 'error' not in response_json else "error",
    error=response_json.get('error')
    )
print("Response object:", response_obj)

## Phase 2: Communication Bridge — Complete
- Implemented and tested `query_lm_studio(prompt)` with error handling and logging.
- Defined and validated JSON message schema for agent requests and responses.
- All API interactions are logged for debugging.

Ready to proceed to Phase 3: Copilot Output Integration.

In [ ]:
import os
COPILOT_OUTPUT_PATH = "c:\\Users\\tyler\\source\\repos\\Wulfic\\Orchestrator\\copilot_output.json"

def read_copilot_output(max_retries=3, retry_delay=2):
    """Read and return the contents of the Copilot output file as a Python object. Retries on failure."""
    for attempt in range(1, max_retries + 1):
        try:
            with open(COPILOT_OUTPUT_PATH, "r", encoding="utf-8") as f:
                return json.load(f)
        except FileNotFoundError:
            logging.error(f"Copilot output file not found: {COPILOT_OUTPUT_PATH} (Attempt {attempt})")
            if attempt < max_retries:
                time.sleep(retry_delay)
            else:
                return None
        except json.JSONDecodeError as e:
            logging.error(f"JSON decode error: {e} (Attempt {attempt})")
            if attempt < max_retries:
                time.sleep(retry_delay)
            else:
                return None
        except Exception as e:
            logging.error(f"Error reading copilot output: {e} (Attempt {attempt})")
            if attempt < max_retries:
                time.sleep(retry_delay)
            else:
                return None

def write_copilot_output(data, max_retries=3, retry_delay=2):
    """Write the given Python object to the Copilot output file as JSON. Retries on failure."""
    for attempt in range(1, max_retries + 1):
        try:
            with open(COPILOT_OUTPUT_PATH, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2)
            logging.info(f"Copilot output written to {COPILOT_OUTPUT_PATH}")
            return True
        except Exception as e:
            logging.error(f"Error writing copilot output: {e} (Attempt {attempt})")
            if attempt < max_retries:
                time.sleep(retry_delay)
            else:
                return False

In [ ]:
# Test writing and reading Copilot output
test_data = {"copilot_output": ["Hello, world!", "Test entry"]}
write_success = write_copilot_output(test_data)
print("Write success:", write_success)
read_data = read_copilot_output()
print("Read data:", read_data)

## Phase 3: Copilot Output Integration — Complete
- Implemented and tested `read_copilot_output()` and `write_copilot_output(data)` functions.
- File I/O operations verified for correct JSON serialization and error handling.
- Ready to standardize file naming conventions and directory structure, then proceed to Phase 4.

In [ ]:
class OrchestratorAgent:
    def __init__(self, state_file="orchestrator_state.json"):
        self.steps = []
        self.current_step = 0
        self.state_file = state_file
        self.load_state()

    def add_step(self, description, action):
        self.steps.append({"description": description, "action": action})
        self.save_state()

    def run_next_step(self, max_retries=3, retry_delay=2):
        if self.current_step < len(self.steps):
            step = self.steps[self.current_step]
            logging.info(f"Running step {self.current_step + 1}: {step['description']}")
            for attempt in range(1, max_retries + 1):
                try:
                    result = step["action"]()
                    logging.info(f"Step output: {result}")
                    self.current_step += 1
                    self.save_state()
                    return result
                except Exception as e:
                    logging.error(f"Error in step {self.current_step + 1} (Attempt {attempt}): {e}")
                    if attempt < max_retries:
                        time.sleep(retry_delay)
                    else:
                        logging.error(f"Step {self.current_step + 1} failed after {max_retries} attempts.")
                        return None
        else:
            logging.info("No more steps to run.")
            return None

    def save_state(self, max_retries=3, retry_delay=2):
        state = {
            "steps": [s["description"] for s in self.steps],
            "current_step": self.current_step
        }
        for attempt in range(1, max_retries + 1):
            try:
                with open(self.state_file, "w", encoding="utf-8") as f:
                    json.dump(state, f, indent=2)
                return
            except Exception as e:
                logging.error(f"Error saving state (Attempt {attempt}): {e}")
                if attempt < max_retries:
                    time.sleep(retry_delay)
                else:
                    logging.error(f"Failed to save state after {max_retries} attempts.")

    def load_state(self, max_retries=3, retry_delay=2):
        if os.path.exists(self.state_file):
            for attempt in range(1, max_retries + 1):
                try:
                    with open(self.state_file, "r", encoding="utf-8") as f:
                        state = json.load(f)
                    self.current_step = state.get("current_step", 0)
                    return
                except Exception as e:
                    logging.error(f"Error loading state (Attempt {attempt}): {e}")
                    if attempt < max_retries:
                        time.sleep(retry_delay)
                    else:
                        logging.error(f"Failed to load state after {max_retries} attempts.")

In [ ]:
# Test OrchestratorAgent with simple steps
def step1():
    print("Step 1: Query LM Studio")
    return query_lm_studio("Write a Python function to add two numbers.")

def step2():
    print("Step 2: Read Copilot Output")
    return read_copilot_output()

agent = OrchestratorAgent()
agent.add_step("Query LM Studio for code generation", step1)
agent.add_step("Read Copilot output", step2)

while agent.current_step < len(agent.steps):
    output = agent.run_next_step()
    print(f"Step {agent.current_step}: Output:", output)

## Phase 4: Orchestrator Agent — Complete
- Created `OrchestratorAgent` class with step management, persistent state, and logging.
- Tested agent with basic workflow steps.
- Ready for Phase 5: Basic Workflow Demo.

In [ ]:
# Phase 5: Basic Workflow Demo
def step_query_lm():
    prompt = "Write a Python function to multiply two numbers."
    response = query_lm_studio(prompt)
    code = response.get('choices', [{}])[0].get('message', {}).get('content', "No output")
    print("LM Studio response:", code)
    # Save to Copilot output file
    write_copilot_output({"copilot_output": [code]})
    return code

def step_read_copilot():
    data = read_copilot_output()
    print("Copilot output read:", data)
    return data

def step_validate_output():
    data = read_copilot_output()
    # Simple validation: check if output contains 'def'
    valid = any('def' in entry for entry in data.get('copilot_output', []))
    print("Validation result:", valid)
    return valid

workflow_agent = OrchestratorAgent()
workflow_agent.add_step("Query LM Studio for multiplication function", step_query_lm)
workflow_agent.add_step("Read Copilot output", step_read_copilot)
workflow_agent.add_step("Validate output contains Python function", step_validate_output)

while workflow_agent.current_step < len(workflow_agent.steps):
    result = workflow_agent.run_next_step()
    print(f"Step {workflow_agent.current_step}: Result:", result)

## Phase 5: Basic Workflow Demo — Complete
- Demonstrated full workflow: querying LM Studio, writing/reading Copilot output, and validating results.
- Workflow steps executed and outputs printed for review.
- Ready for Phase 6: Review & Next Steps.

## Phase 6: Review & Next Steps
- Extend error handling with retry policies and fallback actions for robust workflows.
- Build a lightweight UI or CLI for step approval and feedback.
- Define and track success metrics (linting, test coverage, workflow latency).
- Integrate additional agents (test runners, deployment bots) and multi-language support.

You can now iterate on these improvements to further enhance the Orchestrator system.